In [ ]:
"""
A few decisions.

###########################################################
# Parsing the answer:
###########################################################
We take the answer by taking the last integer in the answer string.
If we don't find an integer, we call word2num on the answer string, to maybe get a word form number.
If we don't find a number, in the majority vote case, we ignore that generation in the vote.
If none of the generations have a number, we return the default value.
Similarily, in the case without majority vote, if we don't find a number, we return the default value.

It would be fair to say that we should maybe not use the default value, and just assign the answer as incorrect.
We may do that eventually. It would be pretty straightfoward, just change the default value to a stopper object,
the comparison with the reference answer would always be false.

Floating point numbers:
We currently round to the closest integer.


###########################################################
# Picking questions in the dataset:
###########################################################
More than 85% of the questions in the ASDiv dataset have a single, integer answer.
We only consider those questions, as they make the task of parsing the answer easier.

"""


import collections

import importlib
import itertools
import math
import more_itertools
from pathlib import Path
import random
import re
import time
import xml

import datasets
import matplotlib.pyplot as plt
import numpy as np
import rich
import rich.table
from text2digits import text2digits
import torch
from tqdm import tqdm
import transformers
import wget

import general_utils as utils

import asdiv_dataset

# importlib.reload(text2digits)

In [ ]:
dataset_train = datasets.load_dataset("gsm8k", "main", split="train")
dataset_test  = datasets.load_dataset("gsm8k", "main", split="test")

def clean_text(sample):
    return {k: v.replace("<<", "(").replace(">>", ")").strip() for k, v in sample.items()}

def split_answer_scratchpad(sample):
    scratchpad, answer = sample["answer"].split("####")
    return {
        "question": sample["question"].strip(), 
        "answer": answer.strip(), 
        "scratchpad": scratchpad.strip()
    }


dataset_train = dataset_train.map(clean_text).map(split_answer_scratchpad)
dataset_test  = dataset_test .map(clean_text).map(split_answer_scratchpad)

print(dataset_train[0].keys())
print(dataset_test[0].keys())


In [ ]:
# dataset_train = asdiv_dataset.ASDivInteger(cache_path="ASDiv.xml", quiet=False)
# dataset_test  = dataset_train

In [ ]:
###############################################################################
# Load the model
###############################################################################
model_name = "google/flan-t5-xxl"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

with utils.ctx_timeit(f"Loading model `{model_name}`"):
    model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_name)

with utils.ctx_timeit("Converting model's type"):
    # model_cpu
    pass

with utils.ctx_timeit(f"Moving model to GPU"):
    model = model.cuda()

rich.print(f"[bold blue]Model dtype:[/]  {model.dtype}")
devices = collections.Counter(x.device.type for x in model.parameters())
rich.print(f"\n[bold blue]Model device:[/] {devices}")

assert len(devices) == 1 and "cuda" in devices, devices

In [5]:

text2digits_ = text2digits.Text2Digits()
num_pat = re.compile(r"\d+(?:[\,\.]\d+)?")


def deal_with_words(text):
    converted = text2digits_.convert(text)
    output = num_pat.findall(converted)[-1]
    rich.print(
        f"[bold blue]text2digits[/]:\n"
        f" \t - [green]source:[/]    {text}\n"
        f" \t - [green]converted:[/] {converted}\n"
        f" \t - [green]final:[/]     {output}"
    )
    return output 


def split_fn(x):
    results = num_pat.findall(x)
    if not results:
        try:
            output = deal_with_words(x)
        except ValueError:
            output = None

        if output is not None:
            rich.print(f"[red]split_fn: no numbers found. Received:[/] `{x}`. Text2Digit worked. Output: `{output}`")
            output = str(output)
        else:
            rich.print(f"[red]split_fn: no numbers found. Received:[/] `{x}`")
            output = None
    else:
        output = results[-1]
    return output


class ContextGeneration:
    """
    
    Namespace

    """

    chain_of_thought_intro = "Let's think about it step by step. Chain-of-thought:"
    answer_intro           = "Answer:"
    question_intro         = "Question:"

    @classmethod
    def compose_fewshot_context(cls, dataset, n, with_scratchpad, seed):
        """ 
        Creates a random few-shot context. Works fine with n = 0.
        """
        rng = random.Random(seed)

        indices = rng.sample(range(len(dataset)), n)
        output = []
        for i in indices:
            scratchpad = dataset[i]["scratchpad"]
            answer = dataset[i]["answer"]

            assert "#" not in scratchpad, scratchpad
            assert "#" not in answer, answer

            text = "Question: " + dataset[i]["question"]
            if with_scratchpad:
                text += f" {cls.chain_of_thought_intro} " + scratchpad
            
            text += f" {cls.answer_intro} " + answer
            output.append(text)

        return " ".join(output)

    @classmethod
    def collate(cls, inputs, tokenizer, few_shot_context, with_scratchpad):
        """ Collates the inputs and prompts into a single list of strings. """

        first_context_addition = few_shot_context + f" {cls.question_intro} "
        final_context_addition = f" {cls.chain_of_thought_intro} " if with_scratchpad else f" {cls.answer_intro} "

        inputs = utils.dict_unzip(inputs)

        question_text = [
            first_context_addition + question + final_context_addition 
            for question in inputs["question"]
        ]

        # rich.print(
        #     f"[bold blue]Question example:[/]\n" +
        #     random.choice(question_text) 
        # )

        output = tokenizer(
            question_text,
            padding=True,
            return_tensors="pt"
        ) 

        output["answer"] = inputs["answer"]
        output["scratchpad"] = inputs["scratchpad"]
        
        return {
            k: v.to("cuda") if isinstance(v, torch.Tensor) else v 
            for k, v in output.items()
    }


###############################################################################
# Dataset stuff
###############################################################################
def format_output(output):
    try:
        float_conversion = float(output.replace(",", ""))
    except (ValueError, ArithmeticError) as err:
        rich.print(
            f"[bold red]Failed to convert to float. "
            f"value is:[/] `{output}`, "
            f"[bold red]error is:[/] {type(err)} {err}"
        )
        return "0"
    
    rounding = round(float_conversion)
    return str(rounding)


def majority_vote(generated, tokenizer, answer_extraction_fn, verbose):
    answers = []
    for entry in generated:
        decoded = tokenizer.decode(entry, skip_special_tokens=True)
        output = answer_extraction_fn(decoded)
        if output is not None:
            answers.append(format_output(output))
    
    counter = collections.Counter(answers)
    if verbose:
        print(counter)
    if counter:
        return counter.most_common(1)[0][0]
    else:
        return "0"


def majority_vote_batch(generated, tokenizer, answer_extraction_fn, verbose):
    for entry in generated:
        yield majority_vote(entry, tokenizer, answer_extraction_fn, verbose)

def compare(pred, answ):
    return format_output(pred.strip()) == format_output(answ.strip())


def run(
    *,
    shuffle,
    verbose,
    context,
    num_beams,
    batch_size,
    max_new_tokens,
    with_scratchpads,
    use_majority_vote,
    use_group_beam_search,
    generation_extra_kwargs
):
    args = locals().copy()
    
    with torch.inference_mode():
        dataloader = torch.utils.data.DataLoader(
            dataset_test,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=lambda inputs: ContextGeneration.collate(
                inputs, tokenizer, context, with_scratchpads
            )
        )

        outputs = []
        tqdm_obj = tqdm(dataloader)

        extra_kwargs = generation_extra_kwargs.copy()
        if use_group_beam_search:
            extra_kwargs["num_beam_groups"] = num_beams

        for batch in tqdm_obj:
            output = model.generate(
                input_ids            = batch["input_ids"],
                attention_mask       = batch["attention_mask"],
                num_beams            = num_beams,
                num_return_sequences = num_beams if use_majority_vote else 1,
                max_new_tokens       = max_new_tokens,
                **extra_kwargs
            ).reshape(batch_size, num_beams if use_majority_vote else 1, -1)

            predictions = list(majority_vote_batch(output, tokenizer, split_fn, verbose))
            raw_decoded = [
                [tokenizer.decode(x, skip_special_tokens=True) for x in batch_entry] 
                for batch_entry in output
            ]

            answer_decoded = [list(map(split_fn, x)) for x in raw_decoded]

            if verbose:
                for prediction, answer, raw_decoded_entry, answer_decoded, input_text in zip(
                    predictions, 
                    batch["answer"], 
                    raw_decoded, 
                    answer_decoded, 
                    [tokenizer.decode(x) for x in batch["input_ids"]]
                ):

                    rich.print(
                        f"[bold blue]input_text[/]:      {input_text}\n"
                        f"[bold blue]ref_answer[/]:      {answer}\n"
                        f"[bold blue]prediction[/]:      {prediction}\n"
                        # f"[bold blue]raw_decoded[/]:     {raw_decoded_entry}\n"
                        f"[bold blue]answer_decoded[/]:  {answer_decoded}"
                    )
                    rich.print("[bold blue]Raw Decoded:[/]")
                    for v in raw_decoded_entry:
                        rich.print(f" [bold]-[/] {v}")

            good_bad_preds = [
                compare(pred=pred, answ=answ) 
                for pred, answ in zip(predictions, batch["answer"])
            ]

            # print([(format_output(a), format_output(b), a, b) for a, b in zip(predictions, batch["answer"])])
            # print(good_bad_preds)

            outputs.extend(good_bad_preds)
            tqdm_obj.set_description(f"Accuracy: {np.mean(outputs):.1%}")
            
        accuracy = np.mean(outputs)

        rich.print(args)
        rich.print(f"[bold green]Accuracy, {model.dtype}: {accuracy:.1%}")


verbose = True
shuffle = True
n_shots = 16
num_beams = 8
batch_size = 1
max_new_tokens = 200
with_scratchpads = True
use_majority_vote = True
use_group_beam_search = False
few_shot_context_rng_seed = 42  # Makes sure the context is the same if we want it to stay the same

generation_extra_kwargs = dict(
    repetition_penalty=50.,
)


context = ContextGeneration.compose_fewshot_context(
        dataset_train, 
        n_shots, 
        with_scratchpads, 
        few_shot_context_rng_seed,
    )


rich.print(
    f"[bold blue]Context[/]:\n" +
    context
)


run(
    shuffle=shuffle,
    verbose=verbose,
    context=context,
    num_beams=num_beams, 
    batch_size=batch_size, 
    max_new_tokens=max_new_tokens, 
    with_scratchpads=with_scratchpads, 
    use_majority_vote=use_majority_vote, 
    use_group_beam_search=use_group_beam_search,
    generation_extra_kwargs=generation_extra_kwargs
)

Context:
Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive 
$1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive? Let's 
think about it step by step. Chain-of-thought: There are 144/12 = (144/12=12)12 sets of 12 cans that the family 
collected.
So, the family would receive $0.50 x 12 = $(0.50*12=6)6 for the cans.
There are 20/5 = (20/5=4)4 sets of 5 kilograms of newspapers that the family collected.
So, the family would receive $1.50 x 4 = $(1.50*4=6)6 for the newspapers.
Therefore, the family would receive a total of $6 + $6 = $(6+6=12)12. Answer: 12 Question: Betty picked 16 
strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. They used their 
strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they
able to make from the strawberries they picked? Let's think about it step by step. Chain-of-thought: Matthew picked
16 + 20 = (16+20=36)36 strawberries.
Natalie picked 36/2 = (36/2=18)18 strawberries.
All together, they have 16 + 36 + 18 = (16+36+18=70)70 strawberries.
They can make 70/7 = (70/7=10)10 jars of strawberries.
They earn 10 x $4 = $(10*4=40)40 from the strawberries they picked. Answer: 40 Question: Jack has a stack of books 
that is 12 inches thick. He knows from experience that 80 pages is one inch thick. If he has 6 books, how many 
pages is each one on average? Let's think about it step by step. Chain-of-thought: There are 960 pages because 80 x
12 = (80*12=960)960
Each book is 160 pages because 960 / 6 = (960/6=160)160 Answer: 160 Question: James dumps his whole collection of 
500 Legos on the floor and starts building a castle out of them.  He uses half the pieces before finishing and is 
told to put the rest away.  He puts all of the leftover pieces back in the box they came from, except for 5 missing
pieces that he can't find.  How many Legos are in the box at the end? Let's think about it step by step. 
Chain-of-thought: James starts with 500 Legos and uses half of them, leaving 500/2=(500/2=250)250 Legos unused.
He puts those unused Legos away but since he's missing 5 he only puts 250-5=(250-5=245)245 Legos away. Answer: 245 
Question: Ines had $20 in her purse. She bought 3 pounds of peaches, which are $2 per pound at the local farmers’ 
market. How much did she have left? Let's think about it step by step. Chain-of-thought: Ines bought 3 pounds of 
peaches for 3 peaches * $2/peach = $(3*2=6)6.
Ines has $20 - $6 = $(20-6=14)14 left. Answer: 14 Question: Aaron pays his actuary membership fees each year. The 
membership fee increases yearly by $10. If he pays $80 in the first year, how much does his membership cost, in 
dollars, in the sixth year? Let's think about it step by step. Chain-of-thought: In year 2 he pays 
80+10=$(80+10=90)90.
In year 3 he pays 90+10=$(90+10=100)100.
In year 4 he pays 100+10=$(100+10=110)110.
In year 5 he pays 110+10=$(110+10=120)120.
In year 6 he pays 120+10=$(120+10=130)130. Answer: 130 Question: Joseph invested $1000 into a hedge fund. The fund 
promised a yearly interest rate of 10%. If he deposited an additional $100 every month into the account to add to 
his initial investment of $1000, how much money will he have in the fund after two years? Let's think about it step
by step. Chain-of-thought: For the first year, Joseph will have invested $1000 + ($100 * 12) = 
$(1000+100*12=2200)2200.
The interest calculated for the first year will be $2200 * 10% = $(2200*10*.01=220)220.
The total value of the investment for the first year will be $2200 + $220 = $(2200+220=2420)2420.
For the second year, the total invested will be $2420 + ($100 * 12) = $(2420+100*12=3620)3620.
The interest calculated after the second year will be $3620 * 10% = $(3620*10*.01=362)362.
Therefore, Joseph's investment in the mutual fund will be worth $3620 + $362 = $(3620+362=3982)3982. Answer: 3982 


  0%|          | 0/1319 [00:00<?, ?it/s]

Counter({'70': 4, '75': 2, '65': 1, '80': 1})


input_text:      Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of 
newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money 
would you receive? Let's think about it step by step. Chain-of-thought: There are 144/12 = (144/12=12)12 sets of 12
cans that the family collected. So, the family would receive $0.50 x 12 = $(0.50*12=6)6 for the cans. There are 
20/5 = (20/5=4)4 sets of 5 kilograms of newspapers that the family collected. So, the family would receive $1.50 x 
4 = $(1.50*4=6)6 for the newspapers. Therefore, the family would receive a total of $6 + $6 = $(6+6=12)12. Answer: 
12 Question: Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as 
Natalie. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4.
How much money were they able to make from the strawberries they picked? Let's think about it step by step. 
Chain-of-thought: Matthew picked 16 + 20 = (16+20=36)36 strawberries. Natalie picked 36/2 = (36/2=18)18 
strawberries. All together, they have 16 + 36 + 18 = (16+36+18=70)70 strawberries. They can make 70/7 = (70/7=10)10
jars of strawberries. They earn 10 x $4 = $(10*4=40)40 from the strawberries they picked. Answer: 40 Question: Jack
has a stack of books that is 12 inches thick. He knows from experience that 80 pages is one inch thick. If he has 6
books, how many pages is each one on average? Let's think about it step by step. Chain-of-thought: There are 960 
pages because 80 x 12 = (80*12=960)960 Each book is 160 pages because 960 / 6 = (960/6=160)160 Answer: 160 
Question: James dumps his whole collection of 500 Legos on the floor and starts building a castle out of them. He 
uses half the pieces before finishing and is told to put the rest away. He puts all of the leftover pieces back in 
the box they came from, except for 5 missing pieces that he can't find. How many Legos are in the box at the end? 
Let's think about it step by step. Chain-of-thought: James starts with 500 Legos and uses half of them, leaving 
500/2=(500/2=250)250 Legos unused. He puts those unused Legos away but since he's missing 5 he only puts 
250-5=(250-5=245)245 Legos away. Answer: 245 Question: Ines had $20 in her purse. She bought 3 pounds of peaches, 
which are $2 per pound at the local farmers’ market. How much did she have left? Let's think about it step by step.
Chain-of-thought: Ines bought 3 pounds of peaches for 3 peaches * $2/peach = $(3*2=6)6. Ines has $20 - $6 = 
$(20-6=14)14 left. Answer: 14 Question: Aaron pays his actuary membership fees each year. The membership fee 
increases yearly by $10. If he pays $80 in the first year, how much does his membership cost, in dollars, in the 
sixth year? Let's think about it step by step. Chain-of-thought: In year 2 he pays 80+10=$(80+10=90)90. In year 3 
he pays 90+10=$(90+10=100)100. In year 4 he pays 100+10=$(100+10=110)110. In year 5 he pays 
110+10=$(110+10=120)120. In year 6 he pays 120+10=$(120+10=130)130. Answer: 130 Question: Joseph invested $1000 
into a hedge fund. The fund promised a yearly interest rate of 10%. If he deposited an additional $100 every month 
into the account to add to his initial investment of $1000, how much money will he have in the fund after two 
years? Let's think about it step by step. Chain-of-thought: For the first year, Joseph will have invested $1000 + 
($100 * 12) = $(1000+100*12=2200)2200. The interest calculated for the first year will be $2200 * 10% = 
$(2200*10*.01=220)220. The total value of the investment for the first year will be $2200 + $220 = 
$(2200+220=2420)2420. For the second year, the total invested will be $2420 + ($100 * 12) = 
$(2420+100*12=3620)3620. The interest calculated after the second year will be $3620 * 10% = $(3620*10*.01=362)362.
Therefore, Joseph's investment in the mutual fund will be worth $3620 + $362 = $(3620+362=3982)3982. 

Raw Decoded:

...

- A pen costs as much as a pencil and eraser combined so that's (1.20 + 0.30) = $1.40 per pen. 8 pens will cost (8
* 1.40) = $24. The answer: 24.

- A pen costs as much as a pencil and eraser combined so that's (1.20 + 0.30) = $1.60 per pen. 8 pens will cost $8
* 1.60 = $64.00 in total. Answer: 64

- A pen costs as much as a pencil and eraser combined so that's (1.20 + 0.30) = $1.40 per pen. 8 pens will cost (8
* 1.40) = $12.00. Answer: 12

Accuracy: 0.0%:   0%|          | 4/1319 [00:47<4:20:47, 11.90s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_385232/2310525958.py:266 in <cell line: 266>                                      │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_385232/2310525958.py'                       │
│                                                                                                  │
│ /tmp/ipykernel_385232/2310525958.py:182 in run                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_385232/2310525958.py'                       │
│                                                                                                  │
│ /home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/torch/autograd/grad_mode.py:27  │
│ in decorate_context                                                                              │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/transformers/generation_utils.p │
│ y:1577 in generate                                                                               │
│                                                                                                  │
│   1574 │   │   │   │   input_ids, expand_size=num_beams, is_encoder_decoder=self.config.is_enco  │
│   1575 │   │   │   )                                                                             │
│   1576 │   │   │   # 12. run beam search                                                         │
│ ❱ 1577 │   │   │   return self.beam_search(                                                      │
│   1578 │   │   │   │   input_ids,                                                                │
│   1579 │   │   │   │   beam_scorer,                                                              │
│   1580 │   │   │   │   logits_processor=logits_processor,                                        │
│                                                                                                  │
│ /home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/transformers/generation_utils.p │
│ y:2806 in beam_search                                                                            │
│                                                                                                  │
│   2803 │   │   │   next_tokens = next_tokens % vocab_size                                        │
│   2804 │   │   │                                                                                 │
│   2805 │   │   │   # stateless                                                                   │
│ ❱ 2806 │   │   │   beam_outputs = beam_scorer.process(                                           │
│   2807 │   │   │   │   input_ids,                                                                │
│   2808 │   │   │   │   next_token_scores,                  

### ASDiv Dataset:

- **At a glance**: float32, no context (except let's do this step by step), 8 beams majority vote, fixed number parser.
  - **Accuracy**: 37.9%
  - **Precision**: float32
  - **num_beams**: 8
  - **batch_size**: 1
  - **max_new_tokens**: 100
  - **use_majority_vote**: True
  - **use_group_beam_search**: False
  - **generation_extra_kwargs**: 
    - *repetition_penalty*: 50.0
    - *context*: ''
  - **Notes**: 
    - For the word outputs, the word2num often doesn't pickup the correct (last) word.
    - Uses "Let's do this step by step".

- **At a glance**: 8 shot (random) context, with scrachpad. The scratchpads are the Formula from the dataset, so they are pretty poor.
  - **Accuracy**: 41.2%
  - **Precision**: float32
  - **num_beams**: 8
  - **batch_size**: 1
  - **max_new_tokens**: 100
  - **use_majority_vote**: True
  - **use_group_beam_search**: False
  - **generation_extra_kwargs**: 
    - *repetition_penalty*: 50.0
    - *context*: 8 examples in the context, WITH SCRATCHPADS.
    - *context seed*: 42
